# Run HelloWorld toy model through CyberGIS Computing Service

This notebook demostrates the basic steps to run the HelloWorld toy model through the CyberGIS Computing Service (CCS). The HelloWorld toy model was created as an example for model developers who want to integrate their own models into CCS. The only required input of this HelloWorld model is a file named "in.txt" where users can put arbitrary multi-line plan text. The model reads in the content of "in.txt" and inserts text "Hello World !" on the first line. It then writes the updated content to file "out.txt". The implementation and instructions on the HelloWorld mode can be found at https://github.com/cybergis/Jupyter-xsede/blob/master/cybergis/helloworld.py. 

In [ ]:
import os
import tempfile
import time

### Create workspace folder

In [ ]:
workspace = "./workspace"

In [ ]:
# create a local model folder named "helloworld_<CURRENT_EPOCH_TIME>"
workspace = os.path.abspath(workspace)
model_folder = "helloworld_" + str(int(time.time()))
model_folder = os.path.join(workspace, model_folder)
! mkdir -p {model_folder}
print(model_folder)

In [ ]:
# write current system time to "in.txt", or other arbitrary multi-line plan text
! echo "Time: $(date)" >> {model_folder}/in.txt
! ls -al {model_folder}

In [ ]:
# check content of in.txt
! cat {model_folder}/in.txt

### Submit to CyberGIS Computing Service for execution

In [ ]:
# import CyberGIS Computing Service SDK
from job_supervisor_client import *

In [ ]:
### Create a session 
##  may need to remove previously generated "job_supervisor_constructor_helloworld.json" when switching servers
## ! rm job_supervisor_constructor_helloworld.json

## if test against production job supervisor server
#session = Session("helloworld", isJupyter=True)

## if test against local dev job supervisor server
session = Session("helloworld", url="127.0.0.1", port=3000, isJupyter=True, protocol='HTTP')

In [ ]:
# upload model folder containing in.txt
job = session.job()
job.upload(model_folder)

In [ ]:
# submit job to Keeling/Virtual Roger, and request 1 CPU
job.submit(payload={
    "node": 1,
    "machine": "keeling"
})

In [ ]:
# monitor job execution
job.events(liveOutput=True)

### Retrive model results

In [ ]:
# download model results
output_zip = job.download(model_folder)

In [ ]:
# unzip results and check
! unzip {output_zip} -d {model_folder}
! ls -al {model_folder}

In [ ]:
# check content of out.txt --- "Hello World!" is inserted to the first line
! cat {model_folder}/out.txt

In [ ]:
# check content of the slurm-X.out file generated by HPC scheduler
! cat {model_folder}/slurm-*.out

### Done